In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [2]:
import re
class Valve:
    def __init__(self, name, flow, connections):
        self.name = name
        self.flow = int(flow)
        self.connections = connections
        if int(flow) == 0:
            self.valve_on = True
        else:
            self.valve_on = False
        
    def __str__(self):
        return f'{self.name} flow {self.flow} and goes to {",".join(self.connections)}'

In [3]:
from copy import deepcopy

class ValveState:
    def __init__(self, location, valves, time_left=30):
        self.cur_loc = location
        self.time_left = time_left
        self.valves = valves
        self.total_flow=0
        self.cur_flow=0
        self.relief_per_min = 0
        
        self.releases = []
        self.nvalves_open = sum([s.valve_on for s  in valves.values()])
        
    def decrement_min(self):
        self.cur_flow += self.relief_per_min
        self.time_left -= 1
        
    def cost_of_state(self):
        cost =0
        for valv in self.valves.values():
            if not valv.valve_on:
                cost +=valv.flow
        return cost + (30-self.time_left)
        
    def next_possible_states(self):
        neigh = deepcopy(self)
        
        neigh.decrement_min()
        neighbor_states=[]
        
        if not neigh.valves[neigh.cur_loc].valve_on:
            neigh.valves[neigh.cur_loc].valve_on = True
            neigh.nvalves_open +=1  
            neigh.relief_per_min  += neigh.valves[neigh.cur_loc].flow
            neigh.total_flow  += neigh.valves[neigh.cur_loc].flow * (self.time_left-1)
            neigh.releases.append((neigh.valves[neigh.cur_loc].flow , (self.time_left-1)))
#            neigh.total_flow  += neigh.valves[neigh.cur_loc].flow * (self.time_left-1)
            neighbor_states.append(neigh)
        
        next_tunnels = self.valves[self.cur_loc].connections
        for loc in next_tunnels:
            new_neigh = deepcopy(self)
            new_neigh.decrement_min()
            new_neigh.cur_loc = loc
            neighbor_states.append(new_neigh)
        return neighbor_states
        
    def __lt__(self, other):
        return (self.time_left,-1*self.total_flow) < (other.time_left,-1*other.total_flow)
        

In [4]:
import heapq
class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, ValveState]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: ValveState, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> tuple:
        return heapq.heappop(self.elements)[1]

In [17]:
from collections import deque

def create_paths(initial_state):
    frontier = deque([]) 
    
    seen =set([])
    ends=[]
    total_valvs = len(initial_state.valves)
    frontier.append(initial_state)
    
    while len(frontier)>0:
        current = frontier.popleft()
        
        if current.time_left == 0 or current.nvalves_open == total_valvs :
            ends.append(current)
#            break
        
        for next_state in current.next_possible_states():
            
            tag = (next_state.cur_loc, next_state.relief_per_min,next_state.nvalves_open)
            if tag not in seen:
                seen.add(tag)
                frontier.append(next_state)
    
    max_rel = max([ e.total_flow for e in ends])
    for e in ends:
        if max_rel == e.total_flow :
            return max_rel, e
#    return [ e.total_flow for e in ends]

In [18]:
def read_file(filename):
    str_format= 'Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? (.*)'
    valves={}
    with open(filename, 'r') as f :
        for line in f.readlines():
            m = re.match(str_format,line)
            vals = m.groups()
            neighs = [x.strip() for x in vals[2].split(',')]
            valves[vals[0]] = Valve(vals[0], vals[1], neighs)
    return valves

In [19]:
def part_one(filename):
    initial_valves = read_file(filename)
    initial_state = ValveState( 'AA', initial_valves)
    max_rel, e = create_paths(initial_state )
    return max_rel

In [20]:
%%time 
part_one('input/test_16.txt')

CPU times: user 148 ms, sys: 5.11 ms, total: 154 ms
Wall time: 153 ms


1631

In [9]:
%%time 
part_one('input/day_16.txt')

CPU times: user 1min 5s, sys: 134 ms, total: 1min 5s
Wall time: 1min 5s


1751

# Part 2

In [50]:
from copy import deepcopy

class ValveStateElephant:
    def __init__(self, location, valves, time_left=26):
        self.cur_loc = [location,location]
        self.time_left = time_left
        self.valves = valves
        self.total_flow=0
        self.cur_flow=0
        self.relief_per_min = 0
        
        self.releases = []
        self.nvalves_open = sum([s.valve_on for s  in valves.values()])
        
    def decrement_min(self):
        self.cur_flow += self.relief_per_min
        self.time_left -= 1
        
    def set_state(self,iloc, jloc):
        self.cur_loc[0] = iloc
        self.cur_loc[1] = jloc
        
    def get_priority(self):
        potential = sum([s.flow for s in self.valves.values() if not s.valve_on])
        return potential/(self.time_left+1)
        
    def next_possible_states(self):
        neigh = deepcopy(self)
        
        neigh.decrement_min()
        neighbor_states=[]
        next_tunnels_i = self.valves[neigh.cur_loc[0]].connections
        next_tunnels_j = self.valves[neigh.cur_loc[1]].connections
        if not neigh.valves[neigh.cur_loc[0]].valve_on:
            next_tunnels_i.append(neigh.cur_loc[0])
            
        if not neigh.valves[neigh.cur_loc[1]].valve_on:
            next_tunnels_j.append(neigh.cur_loc[1])
            
            
        for iloc in next_tunnels_i:
            n = deepcopy(neigh)
            if iloc == neigh.cur_loc[0]:
                n.valves[iloc].valve_on = True
                n.nvalves_open +=1  
                n.relief_per_min  += n.valves[iloc].flow
                n.total_flow  += n.valves[iloc].flow * (self.time_left-1)
                n.releases.append((n.valves[iloc].flow , (self.time_left-1)))
            
            for jloc in next_tunnels_j:
                nn= deepcopy(n)
                if (iloc == jloc) and (self.cur_loc[0]==self.cur_loc[1])  and (iloc == self.cur_loc[0]): 
                    continue
                    
                if jloc == neigh.cur_loc[1]:
                    nn.valves[jloc].valve_on = True
                    nn.nvalves_open +=1  
                    nn.relief_per_min  += nn.valves[jloc].flow
                    nn.total_flow  += nn.valves[jloc].flow * (self.time_left-1)
                    nn.releases.append((nn.valves[jloc].flow , (self.time_left-1)))
                    
                nn.set_state(iloc,jloc)
                neighbor_states.append(nn)
                
        return neighbor_states
    
    def __lt__(self, other):
        return (-1*self.relief_per_min/self.time_left,-1*self.total_flow) < (-1*other.relief_per_min/other.time_left,-1*other.total_flow)
        

In [51]:
from collections import deque

def create_paths_two(initial_state):
    frontier = PriorityQueue() 
    
    seen =set([])
    ends=[]
    total_valvs = len(initial_state.valves)
#    frontier.append(initial_state)
    frontier.put(initial_state,0)
    
    while not frontier.empty():
#        current = frontier.popleft()
        current = frontier.get()
        
        if current.time_left == 0 or current.nvalves_open == total_valvs :
            return current.total_flow
            ends.append(current)
        
        for next_state in current.next_possible_states():
            tag = ('-'.join(next_state.cur_loc), next_state.relief_per_min,next_state.nvalves_open)
            if tag not in seen:
                seen.add(tag)
#                frontier.append(next_state)
                priority = next_state.get_priority()
                frontier.put(next_state,priority)
    
    max_rel = max([ e.total_flow for e in ends])
    for e in ends:
        if max_rel == e.total_flow :
            return max_rel, e
#    return [ e.total_flow for e in ends], None

In [52]:
def part_two(filename):
    initial_valves = read_file(filename)
    initial_state = ValveStateElephant( 'AA', initial_valves)
    max_rel = create_paths_two(initial_state )
    return max_rel

In [53]:
%%time
part_two('input/test_16.txt')
#1707

CPU times: user 71.3 ms, sys: 2.48 ms, total: 73.8 ms
Wall time: 73.6 ms


1572

In [ ]:
%%time
part_two('input/day_16.txt')